# Module Info Caching Example

This notebook demonstrates the module info caching functionality in sagely. We'll see how module analysis results are cached to improve performance for subsequent queries about the same module.


In [1]:
import time
from sagely.langgraph_agent import create_agent, analyze_module
from sagely.cache import ModuleInfoCache

# Create an agent instance
agent = create_agent()


ℹ️ Initializing SageAgent with model: gpt-4
ℹ️ Initializing LangGraph agent with model: gpt-4
ℹ️ Building LangGraph workflow...
✅ LangGraph workflow built successfully
✅ LangGraph agent initialized successfully
✅ SageAgent initialized successfully
ℹ️ Initializing SageAgent with model: gpt-4
ℹ️ Initializing LangGraph agent with model: gpt-4
ℹ️ Building LangGraph workflow...
✅ LangGraph workflow built successfully
✅ LangGraph agent initialized successfully
✅ SageAgent initialized successfully
ℹ️ Initializing LangGraph agent with model: gpt-4
ℹ️ Building LangGraph workflow...
✅ LangGraph workflow built successfully
✅ LangGraph agent initialized successfully


## 1. First Analysis

Let's analyze a module for the first time and measure how long it takes.


In [2]:
# Test module to analyze
test_module = "json"

print(f"First analysis of '{test_module}' module:")
print("-" * 40)

# Time the first analysis (should be slower)
start_time = time.time()
result1 = analyze_module.invoke({"module_name": test_module})
first_analysis_time = time.time() - start_time

print(f"Analysis completed in {first_analysis_time:.3f} seconds")
print(f"Result length: {len(result1)} characters")
print(f"First 200 chars: {result1[:200]}...")


First analysis of 'json' module:
----------------------------------------
ℹ️ Analyzing module 'json'...
📦 Using cached module info for 'json'
Analysis completed in 0.002 seconds
Result length: 3803 characters
First 200 chars: Module: json
Documentation: JSON (JavaScript Object Notation) <https://json.org> is a subset of
JavaScript syntax (ECMA-262 3rd edition) used as a lightweight data
interchange format.

:mod:`json` exp...


## 2. Second Analysis (Using Cache)

Now let's analyze the same module again. This time it should be much faster because it will use the cached results.


In [3]:
# Time the second analysis (should be much faster)
start_time = time.time()
result2 = analyze_module.invoke({"module_name": test_module})
second_analysis_time = time.time() - start_time

print(f"Analysis completed in {second_analysis_time:.3f} seconds")
print(f"Speed improvement: {first_analysis_time/second_analysis_time:.1f}x faster")
print(f"Results identical: {result1 == result2}")


ℹ️ Analyzing module 'json'...
📦 Using cached module info for 'json'
Analysis completed in 0.001 seconds
Speed improvement: 1.7x faster
Results identical: True


## 3. Cache Status Check

Let's verify if our module is indeed cached.


In [4]:
is_cached = agent.is_module_cached(test_module)
print(f"Is '{test_module}' cached? {is_cached}")


Is 'json' cached? True


## 4. Clearing Specific Module Cache

Now let's clear the cache for our test module and verify it's no longer cached.


In [5]:
agent.clear_module_cache(test_module)
is_cached_after_clear = agent.is_module_cached(test_module)
print(f"Is '{test_module}' cached after clearing? {is_cached_after_clear}")


📦 Cleared module cache for 'json'
Is 'json' cached after clearing? False


## 5. Testing with Another Module

Let's test the caching behavior with a different module.


In [6]:
another_module = "os"

# First analysis
start_time = time.time()
result3 = analyze_module.invoke({"module_name": another_module})
first_time = time.time() - start_time

# Second analysis (cached)
start_time = time.time()
result4 = analyze_module.invoke({"module_name": another_module})
second_time = time.time() - start_time

print(f"'{another_module}' first analysis: {first_time:.3f}s")
print(f"'{another_module}' second analysis: {second_time:.3f}s")
print(f"Speed improvement: {first_time/second_time:.1f}x faster")


ℹ️ Analyzing module 'os'...
✅ Successfully analyzed module 'os'
ℹ️ Analyzing module 'os'...
📦 Using cached module info for 'os'
'os' first analysis: 0.002s
'os' second analysis: 0.001s
Speed improvement: 2.6x faster


## 6. Clearing All Module Cache

Finally, let's clear the entire module cache and verify that all modules are uncached.


In [7]:
agent.clear_module_cache()  # Clear all
print("All module cache cleared")

# Verify cache is cleared
is_json_cached = agent.is_module_cached(test_module)
is_os_cached = agent.is_module_cached(another_module)
print(f"Is '{test_module}' cached? {is_json_cached}")
print(f"Is '{another_module}' cached? {is_os_cached}")


📦 Cleared all module cache
All module cache cleared
Is 'json' cached? False
Is 'os' cached? False


## Cache Persistence Demo

Now let's demonstrate that the cache persists between different sessions.


In [8]:
# Create a cache instance
cache = ModuleInfoCache()

test_module = "math"

print(f"1. Analyzing '{test_module}' module...")
result = analyze_module.invoke({"module_name": test_module})
print(f"Analysis completed, result length: {len(result)} characters")

print(f"\n2. Creating new cache instance...")
new_cache = ModuleInfoCache()

print(f"3. Checking if '{test_module}' is cached in new instance...")
cached_result = new_cache.get(test_module)

if cached_result:
    print(f"✅ Cache persists! Result length: {len(cached_result)} characters")
    print(f"Results identical: {result == cached_result}")
else:
    print("❌ Cache does not persist (this shouldn't happen)")

print("\n✅ Demo completed!")
print("The module info caching system is working correctly.")
print("This significantly improves performance for repeated queries about the same modules.")


1. Analyzing 'math' module...
ℹ️ Analyzing module 'math'...
✅ Successfully analyzed module 'math'
Analysis completed, result length: 114 characters

2. Creating new cache instance...
3. Checking if 'math' is cached in new instance...
✅ Cache persists! Result length: 114 characters
Results identical: True

✅ Demo completed!
The module info caching system is working correctly.
This significantly improves performance for repeated queries about the same modules.
